<a href="https://colab.research.google.com/github/ShinAsakawa/ShinAsakawa.github.io/blob/master/2022notebooks/2022_0313Kuznetsov_vae_pytorch.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

---
- source: <https://github.com/schatty/misc/blob/master/vae.py>
- blog: [VAE Careful Walkthrough](https://medium.com/@schatty/vae-careful-walkthrough-5d01e7dbf1ab)
- date: 2022_0313 slightly modified from the previous version
- filename: 2022_0313Kuznetsov_vae_pytorch.ipynb
---

# 自己符号化器ネットワーク
<!-- ## Auto-Encoder Nerual Network -->

<center>
<img src="https://miro.medium.com/max/1400/1*VgCLvv53pHp4aMTywlJ6Rg.png" style="width:77%">
</center>

VAE を見る前に，もっと単純なモデル，一般的な自己符号化器から始める。
自己符号化器は，同じ入力データを出力として予測するモデルである。
符号化器は入力データを受け取り，潜在変数次元へとデータを符号化する (潜在的な次元は通常，元のデータの次元よりはるかに小さい)。
復号化器は，出力されるデータが入力データにできるだけ近くなるように，生成された潜在的な表現を入力と同じ次元に復号する。
自己符号化器はラベルを必要としないので，画質向上 (ノイズ除去)，次元削減などに利用できる。
このモデルはデータを再生成すると言えるが，生成モデルとは言えない。
なぜなら，このモデルは新しいサンプルを生成する確率的な要素を持たず，完全に決定論的であり，ただ入力信号を繰り返すだけだからである。
<!-- Before VAE walkthrough let’s start from simpler model, general autoencoder. 
Autoencoder predicts at the output the same input data. 
It consists of two parts: encoder accepts input data and encode it to the latent dimension, which is typially much less than dimension of original data. 
Decoder decodes produced latent representation to the dimension equals to input’s, in such way that outputted data is as close as possible to the input sample. 
Auto-encoders do not need labels and can be used for improvement of image quality (removing noise), dimensionality reduction etc. 
We can say that this model regenerates data, but it is not a generative model. 
The reason is that model has no stochastic component to generate some novel sample, it is complete deterministic and just repeats input signal. -->

# 変分自己符号化器 variational autoencoders
<!--## Variational Auto-Encoder-->

変分原理は物理学に偏在する概念である。

VAE を始めるにあたって重要な考え方は，自己符号化器の欠損成分に関するものである。
確率的な欠落成分を導入するために、我々は $z$ に制限を加え、サンプルが特定の確率密度関数 $Z$ に従って潜在空間内に分布するようにする。
<!-- The key idea to start with VAE concerns missing component of autoencoder. 
To introduce missing component of stochasticity we add a restriction in $z$, such that our samples are distributed in a latent space following a speicifed probability density function $Z$.
-->

<center>
<img src="https://miro.medium.com/max/4800/1*ODYGssktYLlzX6SdB8wr-Q.png" style="width:66%">
</center>

換言すれば $z$ がある特定の形状を持つように強制し，符号化器で元のサンプルをある分布の下で高次元の特徴に写像し，復号化器で $z$ の知識を考慮に入れて画像を再構成する。
元の画素次元からサンプリングする代わりに，縮約した $z$ からサンプリングする。
すなわち，特定の分布，通常は正規分布からサンプリングし，$z$ をある現実的なサンプルに復号化する。
そして $z$ をサンプリングして，新しい $X$ データ点を生成する。
<!-- In other words we enforce $z$ to have some specific shape, use encoder to map original sample into high-level features under some distribution, and use decoder to reconstruct image, taken into account $z$ knowledge. 
Instead of sampling from original large pixel dimensions we will sample from small $z$ (i.e. sample from some distribution, usually normal) and decode our $z$ into some realistic sample. 
Then we can sample $z$ to generate new $X$ data points.
-->

このモデルでは，決定論的なニューラルネットワークを $f$ (学習過程終了後に決定論的) とし，新しい画像をサンプリングするための非決定論的な $z$ を用意する。
ニューラルネットワークの目的は，生成過程における各 $X$ の確率を最大化するようなパラメータ $\theta$ を学習することである：$\displaystyle P(X)=\int P(X\vert z;\theta)\; P(z)\; dz$
<!-- In such a model we have deterministic neural network as $f$ (determenistic after the training process is done), and non-determenistic $z$ to sample new images. 
The goal of neural network is to learn such parameters $\theta$ to maximise the probability of each $X$ under the generative process: $\displaystyle P(X)=\int P(X\vert z;\theta)\; P(z)\; dz$
-->

上式より，すなわち $P(X\vert z)$ と $P(z)$ を用いた尤度最大化問題となる。
積分を実行する代わりに，$X$ を近似する確率変数 $z$ をサンプリングして $P(z\vert X)$ を作る。
$P(z)$ 全体を計算するには，計算論的なコストが高いからである。
だが，$P(z\vert X)$ は未知である。
そこで，変分推論を用いて，$P(z\vert X)$ を $Q(z\vert X)$ で近似する。
ここでのポイントは，実世界をより良く近似する $Q$ を求めることである。
<!-- 
From the previous equation we have a maximum likelihood problem, where we want to know $P(X\vert z)$ and $P(z)$.
We don’t want to calculate the integral, so let’s introduce $P(z\vert X)$ to sample values from $z$ likely to produce $X$, not the whole $P(z)$ possibilities that is too hard from computational perspective. 
But $P(z\vert X)$ is unknown too yet. 
Now variational inference role is to approximate $P(z\vert X)$ with $Q(z\vert X)$. 
So the key idea behind is to find an approximation function that is good enough to represent the real one (that’s define our optimization problem).
-->


このニューラルネットワークを用いた符号化器を近似し，データ点 $X$ の近似点である $z$ を $Q(z\vert X)$ に従うものとして訓練する。
<!-- The approximated function will be our neural encoder that goes from training datapoints $X$ to the likely $z$ points following $Q(z\vert X)$ which models $P(z\vert X)$ -->

<center>
<img src="https://miro.medium.com/max/4800/1*j9JAXB0iSx2ZEbDrKNM7Vw.png" style="width:66%">
</center>

$Q(z\vert X)$ を計算するために，真の分布 と $P(zver X)$ との KL ダイバージェンスを用いて
<!-- To get $Q(z\vert X)$ we compute KL divergence with the true distribution $P(z\vert X)$ -->

$$
D_{KL}\left[Q(z\vert X)\vert\vert P(z\vert X)\right]=
\sum_z Q(z\vert X)\log\frac{Q(z\vert X)}{P(z\vert X)}=
E\left[\log\frac{Q(z\vert X)}{P(z\vert X)}\right]=
E\left[\log Q(z\vert X) - \log P(z\vert X)\right]
$$

ベイズ則を $P(z\vert X)$ へ適用して:
<!-- applying bayesian rule to the $P(z\vert X)$: -->

$$
= E\left[\log Q(z\vert X)-\log\frac{P(X\vert z)P(z)}{P(X)}\right]\\
= E\left[\log Q(z\vert X) - \left(\log P(X\vert z)+\log P(z) - \log P(X)\right)\right]\\
= E\left[\log Q(z\vert X) - \log P(X\vert z) - \log P(z) + \log P(X)\right]
$$

last term under $E$ gets out of expectation for no dependency over $z$ and moves to the left side of equation

$$
D_{KL}\left[Q(z\vert X)\vert\vert P(z\vert X)\right] - \log P(X) =
E\left[\log Q(z\vert X) + \log P(X\vert z) - \log P(z)\right]
$$

右辺を若干変更して

$$
\log P(X) - D_{KL}\left[Q(z\vert X)\vert\vert P(z\vert X)\right]
= E\left[\log P(X\vert z) - \left(\log Q(z\vert X) - \log P(z)\right)\right]
$$

さて，第二項の期待値を展開して，新しい KL 項を表すと

$$
\begin{aligned}
= & E\left[\log P(X\vert z)\right] - E\left[\left(\log Q(z\vert X) - \log P(z)\right)\right]\\
= & E\left[\log P(X\vert z)\right] - D_{KL}\left[Q(z\vert X)\vert\vert P(z)\right]\\
\end{aligned}
$$

結果として VAE の目的関数は以下:
$$
\log P(X) - D_{KL}\left[Q(z\vert X)\vert\vert P(z\vert X)\right] = E\left[\log P(X\vert z)\right] - D_{KL}\left[Q(z\vert X)\vert\vert P(z)\right]
$$

すなわち，データの対数尤度 ($\log P(X)$) をモデル化するために，計算不可能で非負の近似誤差 ($D_ {\text{KL}} \left[Q(z\vert X)\vert P(z\vert X)\right]$) を考慮しなければならないことになる。
上式右辺は，潜在空間が与えられたときのデータの再構成損失 (ニューラル復号化器の再構成損失) から潜在表現の正則化 (ニューラル符号化器の事前予測) を引いたものに等しくなる。
<!-- which means that to model log likelihood of our data ($\log P(X)$) we have to take into account some not computable and non-negative approximation error ($D_{KL}\left[Q(z\vert X)\vert\vert P(z\vert X)\right]$). 
The right side of equation equals reconstruction loss of our data, given latent space (neural decoder reconstruction loss), minus regularization of our latent representation (neural encoder projects over prior).
-->

次に $Q(z\vert X)$ の形状を定義し，事前分布に対するダイバージェンスを計算する（$X$ サンプルを $z$ の表面上に適切に写像したい)。
その方法として，正規分布に予測モーメント $\mu(X)$ と $\sigma(X)$ を分布させる方法が提案されている。
$Q(z\vert X)$ は $\mathcal{N}(\mu(X),\Sigma(X))$ に  $P(z)$ は $\mathcal{N}(0,1)$ になり，KL ダイバージェンスを以下のように計算することができる。
<!-- The next thing to do is to define $Q(z\vert X)$ shape to compute its divergence against prior (we want map $X$ samples over the surface of $z$ in a proper way). 
The proposed way to do so is distribute over normal distribution with predicted moments $\mu(X)$ and $\Sigma(X)$. 
In such a setup our $Q(z\vert X)$ turns into $N(\mu(X),\Sigma(X))$ and $P(z)$ into $N(0,1)$ which allows us to compute KL-divergence as follows
-->

$$
D_{KL}\left[N\left(\mu(X),\Sigma(X))\vert\vert N(0,1)\right)\right]
=\frac{1}{2}\sum_k \left(\Sigma(X)+\mu^2(X)-1-\log\Sigma(X)\right)
$$

これを踏まえて VAE のセットアップは以下のようになる。
<!-- with that in mind our VAE setup would look like this-->

<center>
<img src="https://miro.medium.com/max/1400/1*1Ih4tif_75UO8cv93z2S6Q.png" style="width:88%">
</center>

このモデルでは，符号化器が事前分布に近い平均と標準偏差を予測し (サンプルから抽出した特徴量は分布に近いはず)，$z$ からサンプリングして符号化器に渡し，サンプルを (多少の誤差はあるが) 再構築する。
その後，符号化器を取り除き，事前分布からだけサンプリングすることができるようになる (符号化器がもっともらしいサンプルを再生成できると仮定して)。
まとめると，変分法では $z$ を新たに標本に投影し，$z$ の元標本から，ある誤差を含みながら標本を得ることができる，ということになる。
この誤差は変分下限，あるいは **ELBO** (Evidence Lower Bound Optimization)と呼ばれる。
<!-- This model makes encoder to predict means and standard deviations that are close to the prior distribution (features extracted from our sample should be close to the distribution), then we sample from $z$, give it to decoder and reconstruct sample (with some error). 
Later we will be able remove an encoder and sample just from the prior distribution, assuming decoder can regenerate plausible samples. 
Summarizing, variational methodology tells that we can project $z$ to newly sample, and also get from $z$ original samples with some error. 
This error noted as lower bound, or **ELBO** (Error Lower Bound Optimization).
-->

しかし，上記設定では，あることが原因で現実のものにならない。
ニューラルネットワークは誤差逆伝播アルゴリズムで学習させるので，勾配が計算可能な要素が必要である。
しかし，勾配と勾配をどのように結びつければいいのかがわからない。
$D_ {KL}\left[\mathcal{N}(\mu(X),\Sigma(X) \vert\vert \mathcal{N}(0,1)\right]$ 
なぜなら，この項は微分不能だからである。 
この問題を回避するには，サンプリング過程を次のようなトリックで微分化することで実現する。
<!-- In the setup above however one thing does not allow us to make it real. 
As neural network is trained with backpropagation algorithm, it requires components with flowing gradients. 
In the same time it is not clear how to connect gradients with 
$D\left[N(\mu(X),\Sigma(X)\vert\vert N(0,1)\right]$ as this term is not a differentiable function. 
The workaround of this issue is accomplished by making the sampling process differential with a following trick
-->

$$
z = \mu(X) +\Sigma^{1/2}(X)\epsilon
$$

これを「再パラメータ化トリック」と呼ぶ。
これにより，変分法によるエンド・ツー・エンドの系の学習が可能になる。
<!-- which called the **reparameterization trick**. 
It allows us to train end-to end system in variational methodology.
-->

<center>
<img src="https://miro.medium.com/max/1400/1*dFWLatzOFe84QFjOK7GZ8w.png" style="width:77%">
</center>

モデルの学習が終われば，$z$ の平均と標準偏差をランダムに選んで新しいサンプルを生成し，それを復号化する (エンコーダ成分を破棄する) ことができる。
<!-- After the model is trained we can generate new samples by choosing random mean and standard deviation for $z$ and decode it (discarding the encoder component).
 -->

In [ ]:
import os
import numpy as np
import torch
from torch import nn, optim
from torch.nn import functional as F
from torchvision import datasets, transforms
from torchvision.utils import save_image

device = torch.device("cuda:0") if torch.cuda.is_available() else torch.device("cpu")

In [ ]:
class VAE(nn.Module):
    def __init__(self, data_dim=784, z_dim=10, hidden_dim=500):
        """
        VAE basic model.
        Args:
            data_dim (int): dimension of flatten input
            z_dim (int): dimension of manifold
            hidden_dim (int): dimension of hidden layers between input and manifold
        """
        super(VAE, self).__init__()

        self.fc1 = nn.Linear(data_dim, hidden_dim)
        self.hidden2mu = nn.Linear(hidden_dim, z_dim)
        self.hidden2log_var = nn.Linear(hidden_dim, z_dim)
        self.fc3 = nn.Linear(z_dim, hidden_dim)
        self.fc4 = nn.Linear(hidden_dim, data_dim)
        self.sigmoid = nn.Sigmoid()

    def encode(self, x):
        h1 = F.relu(self.fc1(x))
        return self.hidden2mu(h1), self.hidden2log_var(h1)

    def decode(self, z):
        h3 = F.relu(self.fc3(z))
        return self.sigmoid(self.fc4(h3))

    def reparam(self, mu, log_var):
        std = torch.exp(0.5*log_var)
        eps = torch.randn_like(std)
        return mu + eps * std

    def forward(self, x):
        mu, log_var = self.encode(x.view(-1, 784))
        z = self.reparam(mu, log_var)
        return self.decode(z), mu, log_var


In [ ]:
def train(data_loader, model, loss_func, epoch):
    model.train()
    train_loss = 0
    for batch_i, (data, _) in enumerate(data_loader):
        data = data.to(device)
        optimizer.zero_grad()
        recon_batch, mu, log_var = model(data)
        loss = loss_func(recon_batch, data, mu, log_var)
        loss.backward()
        train_loss += loss.item()
        optimizer.step()

        if batch_i % 100 == 0:
            print(f'訓練エポック:{epoch:02d} ',
                  f'{batch_i * len(data):05d}/{len(data_loader.dataset):05d}',
                  f'({100. * batch_i / len(data_loader):.0f}%)\t',
                  f'損失: {loss.item()/len(data):.3f}')
            
    print(f'====> エポック: {epoch}', 
          f'平均損失: {train_loss/len(data_loader.dataset):.4f}')

In [ ]:
def test(data_loader, model, loss_func):
    model.eval()
    test_loss = 0
    with torch.no_grad():
        for i, (data, _) in enumerate(data_loader):
            data = data.to(device)
            recon_batch, mu, log_var = model(data)
            test_loss += loss_func(recon_batch, data, mu, log_var).item()

    test_loss /= len(data_loader.dataset)
    print('====> Test set loss: {:.4f}'.format(test_loss))

In [ ]:
np.random.seed(42)
torch.manual_seed(42)

config = {
    'epochs': 50,
    'z_dim': 20,
    'input_dim': 784,
    'hidden_dim': 200,
    'batch_size': 128,
    'lr': 0.001,
}
epochs = config['epochs']
batch_size = config['batch_size']
input_dim = config['input_dim']
z_dim = config['z_dim']
hidden_dim = config['hidden_dim']
lr = config['lr']

# Create directory for resulting images
if not os.path.exists('results/reconstruction'):
    os.makedirs('results/reconstruction')

model = VAE(input_dim, z_dim, hidden_dim).to(device)
optimizer = optim.Adam(model.parameters(), lr=lr)

def loss_func(x_reconstructed, x, mu, log_var):
    """損失関数
    引数:
        x_reconstructed (torch.Tenor): decoder output [batch_size, input_size]
        x (torch.Tensor): input data [batch_size, input_size]
        mu (torch.Tensor): [batch_size, z_dim]
        log_var (torch.Tensor): [batch_size, z_dim]

    戻り値 (torch.Tensor): tensor of single loss value
    """
    # 交差エントロピーを再構成誤差として用いる
    bce = F.binary_cross_entropy(x_reconstructed, x.view(-1, input_dim), reduction="sum")
    
    # KL ダイバージェンス
    kld = -0.5 * torch.sum(1 + log_var - mu.pow(2) - log_var.exp())
    return bce + kld

# データの読み込み
train_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', 
                   train=True, 
                   download=True,
                   transform=transforms.ToTensor()),
    batch_size=batch_size, shuffle=True)

test_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', 
                   train=False,
                   transform=transforms.ToTensor()),
    batch_size=batch_size, shuffle=True)

In [ ]:
# 訓練の実施
for epoch in range(1, epochs+1):
    train(train_loader, model, loss_func, epoch)
    test(test_loader, model, loss_func)
    with torch.no_grad():
        sample = torch.randn(20, z_dim).to(device)
        sample = model.decode(sample).cpu()
        save_image(sample.view(20, 1, 28, 28), f'results/sample_{epoch}.png', nrow=10)